In [1]:
"""
curl -s https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.262/aws-java-sdk-bundle-1.12.262.jar \
    -Lo /home/matt/repos/teehr-spark-iceberg/.venv/lib64/python3.10/site-packages/pyspark/jars/aws-java-sdk-bundle-1.12.262.jar

curl -s https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar \
    -Lo /home/matt/repos/teehr-spark-iceberg/.venv/lib64/python3.10/site-packages/pyspark/jars/hadoop-aws-3.3.4.jar

"""
from pyspark.sql import SparkSession
from pyspark import SparkConf


In [2]:
SparkSession.builder.master("local[*]").getOrCreate().stop()

conf = (
    SparkConf()
    .setAppName('TEEHR')
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")
)
## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark.sparkContext.getConf().getAll()

24/06/06 11:33:46 WARN Utils: Your hostname, matt-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/06/06 11:33:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/06 11:33:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.app.startTime', '1717688028519'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', '10

In [3]:
obs = spark.read.parquet("s3a://ciroh-rti-public-data/teehr/protocols/science-eval/timeseries/usgs*.parquet")
obs.show(5)

24/06/06 11:33:49 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+-------------+-------------------+-------------------+---------+-------------+----------------+--------------+
|  location_id|     reference_time|         value_time|    value|variable_name|measurement_unit| configuration|
+-------------+-------------------+-------------------+---------+-------------+----------------+--------------+
|usgs-01010000|2016-01-01 05:00:00|2016-01-01 05:00:00| 64.84558|   streamflow|             cms|usgs_gage_data|
|usgs-01010000|2016-01-01 17:00:00|2016-01-01 17:00:00|61.164387|   streamflow|             cms|usgs_gage_data|
|usgs-01010000|2016-01-02 05:00:00|2016-01-02 05:00:00|58.049534|   streamflow|             cms|usgs_gage_data|
|usgs-01010000|2016-01-02 17:00:00|2016-01-02 17:00:00| 55.21785|   streamflow|             cms|usgs_gage_data|
|usgs-01010000|2016-01-03 05:00:00|2016-01-03 05:00:00|52.669334|   streamflow|             cms|usgs_gage_data|
+-------------+-------------------+-------------------+---------+-------------+----------------+--------

In [4]:
obs.count()

176063841

In [5]:
sim = spark.read.parquet("s3a://ciroh-rti-public-data/teehr/protocols/science-eval/timeseries/nwm2*.parquet")

In [6]:
sim.count()

399505152

In [7]:

xw = spark.read.parquet("s3a://ciroh-rti-public-data/teehr/common/geo/usgs_nwm2*_crosswalk.conus.parquet")

In [8]:
xw.count()

22835

In [9]:
obs.createTempView("obs_temp")
sim.createTempView("sim_temp")
xw.createTempView("xw_temp")

In [10]:
sdf = spark.sql("""SELECT count(*) FROM obs_temp;""")
sdf.show()

+---------+
| count(1)|
+---------+
|176063841|
+---------+

